So, this is the next challenge from Shopee Code League about Logistics Issue.
In terms of host's privacy, I wouldn't share the datasets both training and test. I took the data directly from my Google Drive as I have saved them there. After all, I only needed to mount the drive through Google Colab.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Let's start processing the dataset. At first, as usual, we need to import library from pandas and numpy. Then, we read the csv file to begin our work. 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv("/content/drive/My Drive/Shopee/delivery_orders_march.csv")
df['selleraddress'] #let's see one of the columns inside, selleraddres

0          Pantranco vill. 417 Warehouse# katipunan 532 (...
1          BLDG 210A Moras C42B 2B16,168 church) Complex ...
2          #66 150-C, DRIVE, Milagros Joe socorro Metro M...
3          999maII 201,26 Villaruel Barretto gen.t number...
4          G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...
                                 ...                        
3176308    Intimate St) compound. Sekiat 2B8 98-B Mel far...
3176309    13th 3616 Pilipinas, :1v26 1f-5 (Village) bl10...
3176310    Lane, 379B 1739A Ivc AFPFC Cainta, Waterpark, ...
3176311    lrt-2 Consolacion 224-A Paradise 6D 36st champ...
3176312    (Jeepney Kaligayan VANVAN Salud ASA Lauan A-55...
Name: selleraddress, Length: 3176313, dtype: object

In [ ]:
df.head() # let's take a look on the first five data, there are 6 features on it.

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [ ]:
df.info() # let's see the type of each column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3176313 entries, 0 to 3176312
Data columns (total 6 columns):
 #   Column               Dtype  
---  ------               -----  
 0   orderid              int64  
 1   pick                 int64  
 2   1st_deliver_attempt  float64
 3   2nd_deliver_attempt  float64
 4   buyeraddress         object 
 5   selleraddress        object 
dtypes: float64(2), int64(2), object(2)
memory usage: 145.4+ MB


In [ ]:
df.isna().sum() #now, we can calculate the number of missing values in the data set, and we find out that only 2nd_deliver_attempt column contains NaN value

orderid                      0
pick                         0
1st_deliver_attempt          0
2nd_deliver_attempt    1819311
buyeraddress                 0
selleraddress                0
dtype: int64

First of all, I would like to extract the city names of the order activities from both buyeraddress and selleraddress. If you notice it, all the city names are located at the end of every address. So, I apply both of those columns this split function to extract the city and also convert it into lower-case word.

In [ ]:
def split(x):
  arr=[]
  for i in range(len(x)):
    arr.append((x[i].split(' ')[-1]).lower())
  return arr

In [ ]:
df['buyeraddress']=split(df['buyeraddress'].values)
df['selleraddress']=split(df['selleraddress'].values)

In [ ]:
df['selleraddress'].value_counts() # now we want to count the cities' value and surprisingly they are dominated by manila, with one appearance of luzon

manila    3176312
luzon           1
Name: selleraddress, dtype: int64

Next, I would like to convert those epoch time into the local time by applying pick and 1st_deliver_attempt to 'time' function and then doing extraction only to the date of the delivery. 

In [ ]:
import time
def waktu(x):
  arr=[]
  for i in range(len(x)):
    arr.append((time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x[i])).split(' ')[0]))
  return arr

In [ ]:
df['1st_deliver_attempt']=waktu(df['1st_deliver_attempt'].values)

In [ ]:
df['pick']=waktu(df['pick'].values)

In [ ]:
df.head() #here is our latest dataset after some preprocessing methods

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,2020-03-02,2020-03-05,NaN,manila,manila
1,2219624609,2020-03-04,2020-03-06,1.583799e+09,manila,manila
2,2220979489,2020-03-04,2020-03-06,NaN,manila,manila
3,2221066352,2020-03-05,2020-03-07,NaN,manila,manila
4,2222478803,2020-03-04,2020-03-06,NaN,luzon,manila


Now, let's put more attention to 2nd_deliver_attempt as that is the only column containing missing value. So I create new column, second, to check for any missing values on that column. If it is NaN, second values 0, otherwise 1.

In [ ]:
second=[]
for i in range(len(df['pick'])):
  if df['2nd_deliver_attempt'][i]==np.nan:
    second.append(0)
  else:
    second.append(1)

In [ ]:
df['second']=second

In [ ]:
x=df.loc[df['2nd_deliver_attempt'].isna()]
x['second']=0
df.loc[df['2nd_deliver_attempt'].isna()]=x

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df.head() #print out first 5 data

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,second
0,2215676524,2020-03-02,2020-03-05,NaN,manila,manila,0
1,2219624609,2020-03-04,2020-03-06,1.583799e+09,manila,manila,1
2,2220979489,2020-03-04,2020-03-06,NaN,manila,manila,0
3,2221066352,2020-03-05,2020-03-07,NaN,manila,manila,0
4,2222478803,2020-03-04,2020-03-06,NaN,luzon,manila,0


2nd_deliver_attempt requires special treatment on the column. As it is mixed between NaN and epoch time, we need to apply a function to help us convert the epoch time to local time without violating the NaN values. So, I fill the missing values with 0 first. Then inside the function, if the epoch time is 0, then it return back to NaN. Otherwise, it converts the time into local time and only take the first part of the time, that is the date

In [ ]:
import time
def waktu(x):
  arr=[]
  for i in range(len(x)):
        if(x[i]==0):
            arr.append(np.nan)
        else:
            arr.append((time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x[i])).split(' ')[0]))
  return arr
df['2nd_deliver_attempt']=waktu((df['2nd_deliver_attempt'].fillna(0.000000e+00)).values)

In [ ]:
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,second
0,2215676524,2020-03-02,2020-03-05,NaN,manila,manila,0
1,2219624609,2020-03-04,2020-03-06,2020-03-10,manila,manila,1
2,2220979489,2020-03-04,2020-03-06,NaN,manila,manila,0
3,2221066352,2020-03-05,2020-03-07,NaN,manila,manila,0
4,2222478803,2020-03-04,2020-03-06,NaN,luzon,manila,0


In [ ]:
from datetime import timedelta

Before proceeding to the next step, don't forget to change the type of pick, 1st_deliver_attempt, and 2nd_deliver_attempt into date-time, so that it can help us doing time-series extraction

In [ ]:
df['pick']=pd.to_datetime(df['pick'])
df['1st_deliver_attempt']=pd.to_datetime(df['1st_deliver_attempt'])
df['2nd_deliver_attempt']=pd.to_datetime(df['2nd_deliver_attempt'])

In [ ]:
# (df['1st_deliver_attempt'][0]-df['pick'][0]).days

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3176313 entries, 0 to 3176312
Data columns (total 7 columns):
 #   Column               Dtype         
---  ------               -----         
 0   orderid              int64         
 1   pick                 datetime64[ns]
 2   1st_deliver_attempt  datetime64[ns]
 3   2nd_deliver_attempt  datetime64[ns]
 4   buyeraddress         object        
 5   selleraddress        object        
 6   second               int64         
dtypes: datetime64[ns](3), int64(2), object(2)
memory usage: 169.6+ MB


In [ ]:
df.head() #we can see that three-containing-date columns have been changed into date-time type.

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,second
0,2215676524,2020-03-02,2020-03-05,NaT,manila,manila,0
1,2219624609,2020-03-04,2020-03-06,2020-03-10,manila,manila,1
2,2220979489,2020-03-04,2020-03-06,NaT,manila,manila,0
3,2221066352,2020-03-05,2020-03-07,NaT,manila,manila,0
4,2222478803,2020-03-04,2020-03-06,NaT,luzon,manila,0


Next, we will work with data containing the route from one city to another. First, we need to count the number of delivery from each city to another cities. The route of those cities are given in the delivery matrix (not going to share it to respect the host)

In [ ]:
df.groupby(['buyeraddress','selleraddress'])['orderid'].count()

buyeraddress  selleraddress
luzon         luzon                  1
              manila            804887
manila        manila           1560698
mindanao      manila            331132
visayas       manila            479595
Name: orderid, dtype: int64

In [ ]:
df['route']=3 #we set all the route to be 3 first
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,second,route
0,2215676524,2020-03-02,2020-03-05,NaT,manila,manila,0,3
1,2219624609,2020-03-04,2020-03-06,2020-03-10,manila,manila,1,3
2,2220979489,2020-03-04,2020-03-06,NaT,manila,manila,0,3
3,2221066352,2020-03-05,2020-03-07,NaT,manila,manila,0,3
4,2222478803,2020-03-04,2020-03-06,NaT,luzon,manila,0,3


The next several lines will adjust the correct route from one city to another based on the delivery matrix.

In [ ]:
a=df.loc[(df['buyeraddress']=='luzon')&(df['selleraddress']=='luzon')]
a['route']=5
df.loc[(df['buyeraddress']=='luzon')&(df['selleraddress']=='luzon')]=a
df.loc[(df['buyeraddress']=='luzon')&(df['selleraddress']=='luzon')]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,second,route
2300637,30722935142905,2020-03-23,2020-03-28,NaT,luzon,luzon,0,5


In [ ]:
a=df.loc[(df['buyeraddress']=='luzon')&(df['selleraddress']=='manila')]
a['route']=5
df.loc[(df['buyeraddress']=='luzon')&(df['selleraddress']=='manila')]=a

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
a=df.loc[(df['buyeraddress']=='mindanao')&(df['selleraddress']=='manila')]
a['route']=7
df.loc[(df['buyeraddress']=='mindanao')&(df['selleraddress']=='manila')]=a

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
a=df.loc[(df['buyeraddress']=='visayas')&(df['selleraddress']=='manila')]
a['route']=7
df.loc[(df['buyeraddress']=='visayas')&(df['selleraddress']=='manila')]=a

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df['route'].value_counts() # count the number of each route

3    1560698
7     810727
5     804888
Name: route, dtype: int64

Next, we are going to calculate the actual route based on the dataset and compare it to the route we have set previously based on the matrix. Besides, we need to consider the public holiday and non-working days (Sunday) in the calculation. From this calculation, we can determine whether an order is considered late or not (based on the first and second delivery attempt)

In [ ]:
h1=pd.to_datetime('2020-03-08')
h2=pd.to_datetime('2020-03-25')
h3=pd.to_datetime('2020-03-30')
h4=pd.to_datetime('2020-03-31')
h5=pd.to_datetime('2020-03-01')
h6=pd.to_datetime('2020-03-15')
h7=pd.to_datetime('2020-03-22')
h8=pd.to_datetime('2020-03-29')
h9=pd.to_datetime('2020-04-05')

Here we calculate the duration of the first delivery considering the public holidays and non-working days. We save the result to the newly created column, day_first. If the result is less than (or equal to) the actual route, it is not considered late and otherwise. The late results are kept in the column 'late_first'

In [ ]:
dur=[]
for i in range(len(df['pick'])):
  temp=(df['1st_deliver_attempt'][i]-df['pick'][i]).days
  if (df['pick'][i]<h1) and (h1<df['1st_deliver_attempt'][i]):
    temp=temp-1
  if (df['pick'][i]<h2) and (h2<df['1st_deliver_attempt'][i]):
    temp=temp-1
  if (df['pick'][i]<h3) and (h3<df['1st_deliver_attempt'][i]):
    temp=temp-1
  if (df['pick'][i]<h4) and (h4<df['1st_deliver_attempt'][i]):
    temp=temp-1
  if (df['pick'][i]<h5) and (h5<df['1st_deliver_attempt'][i]):
    temp=temp-1
  if (df['pick'][i]<h6) and (h6<df['1st_deliver_attempt'][i]):
    temp=temp-1
  if (df['pick'][i]<h7) and (h7<df['1st_deliver_attempt'][i]):
    temp=temp-1
  if (df['pick'][i]<h8) and (h8<df['1st_deliver_attempt'][i]):
    temp=temp-1
  if (df['pick'][i]<h9) and (h9<df['1st_deliver_attempt'][i]):
    temp=temp-1
  dur.append(temp)
dur

In [ ]:
df['day_first']=dur

In [ ]:
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,second,route,day_first
0,2215676524,2020-03-02,2020-03-05,NaT,manila,manila,0,3,3
1,2219624609,2020-03-04,2020-03-06,2020-03-10,manila,manila,1,3,2
2,2220979489,2020-03-04,2020-03-06,NaT,manila,manila,0,3,2
3,2221066352,2020-03-05,2020-03-07,NaT,manila,manila,0,3,2
4,2222478803,2020-03-04,2020-03-06,NaT,luzon,manila,0,5,2


In [ ]:
late=[]
for i in range(len(df['pick'])):
  if df['day_first'][i]<=df['route'][i]:
    late.append(0)
  else:
    late.append(1)
late

In [ ]:
df['late_first']=late

In [ ]:
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,second,route,day_first,late_first
0,2215676524,2020-03-02,2020-03-05,NaT,manila,manila,0,3,3,0
1,2219624609,2020-03-04,2020-03-06,2020-03-10,manila,manila,1,3,2,0
2,2220979489,2020-03-04,2020-03-06,NaT,manila,manila,0,3,2,0
3,2221066352,2020-03-05,2020-03-07,NaT,manila,manila,0,3,2,0
4,2222478803,2020-03-04,2020-03-06,NaT,luzon,manila,0,5,2,0


Now, we work with the second delivery attempt. In order to ease us, let's work with non-NaN values first. Just like the first delivery attempt we have done before, we need to calculate the duration between first and second attempt, excluding the public holidays and Sunday, and I put it in 'day_two' column. Next step is compare the values of day_two with 3, if they are greater than 3, they are considered late and otherwise. The results are kept in 'late_second'

In [ ]:
df_sec=df.loc[df['second']==1]
df_sec.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,second,route,day_first,late_first
1,2219624609,2020-03-04,2020-03-06,2020-03-10,manila,manila,1,3,2,0
6,2222738456,2020-03-02,2020-03-05,2020-03-09,manila,manila,1,3,3,0
8,2224704587,2020-03-04,2020-03-05,2020-03-09,luzon,manila,1,5,1,0
16,2227771109,2020-03-02,2020-03-04,2020-03-05,manila,manila,1,3,2,0
17,2227783105,2020-03-02,2020-03-05,2020-03-07,manila,manila,1,3,3,0


In [ ]:
dur=[]
for i in list(df_sec.index):
  temp=(df_sec['2nd_deliver_attempt'][i]-df_sec['1st_deliver_attempt'][i]).days
  if (df_sec['1st_deliver_attempt'][i]<h1) and (h1<df_sec['2nd_deliver_attempt'][i]):
    temp=temp-1
  if (df_sec['1st_deliver_attempt'][i]<h2) and (h2<df_sec['2nd_deliver_attempt'][i]):
    temp=temp-1
  if (df_sec['1st_deliver_attempt'][i]<h3) and (h3<df_sec['2nd_deliver_attempt'][i]):
    temp=temp-1
  if (df_sec['1st_deliver_attempt'][i]<h4) and (h4<df_sec['2nd_deliver_attempt'][i]):
    temp=temp-1
  if (df_sec['1st_deliver_attempt'][i]<h5) and (h5<df_sec['2nd_deliver_attempt'][i]):
    temp=temp-1
  if (df_sec['1st_deliver_attempt'][i]<h6) and (h6<df_sec['2nd_deliver_attempt'][i]):
    temp=temp-1
  if (df_sec['1st_deliver_attempt'][i]<h7) and (h7<df_sec['2nd_deliver_attempt'][i]):
    temp=temp-1
  if (df_sec['1st_deliver_attempt'][i]<h8) and (h8<df_sec['2nd_deliver_attempt'][i]):
    temp=temp-1
  if (df_sec['1st_deliver_attempt'][i]<h9) and (h9<df_sec['2nd_deliver_attempt'][i]):
    temp=temp-1
  dur.append(temp)
dur

In [ ]:
df_sec['day_two']=dur

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_sec.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,second,route,day_first,late_first,day_two
1,2219624609,2020-03-04,2020-03-06,2020-03-10,manila,manila,1,3,2,0,3
6,2222738456,2020-03-02,2020-03-05,2020-03-09,manila,manila,1,3,3,0,3
8,2224704587,2020-03-04,2020-03-05,2020-03-09,luzon,manila,1,5,1,0,3
16,2227771109,2020-03-02,2020-03-04,2020-03-05,manila,manila,1,3,2,0,1
17,2227783105,2020-03-02,2020-03-05,2020-03-07,manila,manila,1,3,3,0,2


In [ ]:
late=[]
for i in list(df_sec.index):
  if df_sec['day_two'][i]<=3:
    late.append(0)
  else:
    late.append(1)
late

In [ ]:
df_sec['late_second']=late

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_sec.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,second,route,day_first,late_first,day_two,late_second
1,2219624609,2020-03-04,2020-03-06,2020-03-10,manila,manila,1,3,2,0,3,0
6,2222738456,2020-03-02,2020-03-05,2020-03-09,manila,manila,1,3,3,0,3,0
8,2224704587,2020-03-04,2020-03-05,2020-03-09,luzon,manila,1,5,1,0,3,0
16,2227771109,2020-03-02,2020-03-04,2020-03-05,manila,manila,1,3,2,0,1,0
17,2227783105,2020-03-02,2020-03-05,2020-03-07,manila,manila,1,3,3,0,2,0


In [ ]:
df.loc[df['second']==1] = df_sec

Last, let's work the NaN values on the 2nd_delivery_attempt column. If the value is NaN, it means that the delivery is done in the first attempt and no second attempt is needed. So, let's put -1 in the late_second for the remaining rows which finished its delivery in the first attempt.

In [ ]:
df['late_second']=-1
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,second,route,day_first,late_first,late_second
0,2215676524,2020-03-02,2020-03-05,NaT,manila,manila,0,3,3,0,-1
1,2219624609,2020-03-04,2020-03-06,2020-03-10,manila,manila,1,3,2,0,-1
2,2220979489,2020-03-04,2020-03-06,NaT,manila,manila,0,3,2,0,-1
3,2221066352,2020-03-05,2020-03-07,NaT,manila,manila,0,3,2,0,-1
4,2222478803,2020-03-04,2020-03-06,NaT,luzon,manila,0,5,2,0,-1


We need to re-calculate the non-NaN cases of second delivery attempt in late_second because all the values are shifted to be -1.

In [ ]:
j=0
for i in list(df_sec.index):
  df['late_second'][i]=late[j]
  j=j+1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


One last step! We now should calculate the final result whether it is late or not. So, basically if the deliveries are done in the first attempt, they are not late and value with the same as the late_first value. Otherwise, if they proceed to the next delivery attempt, it is considered late and the results are the OR-operation on late_first and late_second.

In [ ]:
late=[]
for i in range(len(df['pick'])):
  if (df['late_second'][i]==-1):
    late.append(df['late_first'][i])
  else:
    late.append(df['late_first'][i] or df['late_second'][i])
late

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,


In [ ]:
df['is_late']=late
df

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,second,route,day_first,late_first,late_second,is_late
0,2215676524,2020-03-02,2020-03-05,NaT,manila,manila,0,3,3,0,-1,0
1,2219624609,2020-03-04,2020-03-06,2020-03-10,manila,manila,1,3,2,0,0,0
2,2220979489,2020-03-04,2020-03-06,NaT,manila,manila,0,3,2,0,-1,0
3,2221066352,2020-03-05,2020-03-07,NaT,manila,manila,0,3,2,0,-1,0
4,2222478803,2020-03-04,2020-03-06,NaT,luzon,manila,0,5,2,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3176308,31504087640510,2020-04-02,2020-04-04,NaT,manila,manila,0,3,2,0,-1,0
3176309,31504147352227,2020-04-03,2020-04-06,2020-04-07,manila,manila,1,3,2,0,0,0
3176310,31504462290482,2020-04-02,2020-04-04,NaT,manila,manila,0,3,2,0,-1,0
3176311,31504851495943,2020-04-03,2020-04-07,NaT,luzon,manila,0,5,3,0,-1,0


In [ ]:
submit=df[['orderid','is_late']]

In [ ]:
submit.to_csv('submission3.csv',index=False)